## Launch Script (Extractions) for Inpatient Envelope - GBD 2020
### Survey Extraction, Data Processing, Admin Data Prep

## Setup

In [ ]:
# where your code lives
home_dir = 'FILEPATH'
# your project
proj = 'proj_hospital'
# where you want data written
write_dir ='FILEPATH'

In [ ]:
import pandas as pd
import numpy as np
import os as os

import requests
import copy as copy
import glob as glob

from multiprocessing import Pool

from extractor import gdoc_query, extractor
from db_queries import get_population, get_covariate_estimates, get_model_results
from functions_to_prep_extractions import *
from cluster_utilities import *
from elmo import run

## Read in and prep data for extraction

In [ ]:
# Read in ALL extraction templates - new and old
# Custom function - see extractor.py

# Oldest extractions - "Pre GBD 2017"
pre_gbd_2017 = gdoc_query('FILEPATH')

# GBD 2017 extractions
new_extraction = gdoc_query('FILEPATH')
# indonesia subnational
idn_sub = gdoc_query('FILEPATH')
# india subnational
#ind_sub = gdoc_query('FILEPATH')
# WHO MCCS
mccs = gdoc_query('FILEPATH')
# world health survey
whs = gdoc_query('FILEPATH')
# world bank
wb = gdoc_query('FILEPATH')
# mexico subnational
mex_2006 = gdoc_query('FILEPATH')
# brazil data
bra = gdoc_query('FILEPATH')

# Testing
#tester = gdoc_query('FILEPATH')

# Full 2020 Extraction Sheets
all_2020_extractions = gdoc_query('FILEPATH')
india_2020 = gdoc_query('FILEPATH')

In [ ]:
# Compile all data from above
df_reader = pd.concat([pre_gbd_2017,new_extraction,idn_sub,mccs,whs,wb,mex_2006,bra,all_2020_extractions,india_2020], 
                      sort=False)

In [ ]:
# Take a peek at your data
df_reader

In [ ]:
# SET UP FOLDERS FOR EXTRACTED RAW DATA
# Removes file if it already exists
!rm -r FILEPATH

# Creates new file
!mkdir FILEPATH

In [ ]:
# Subset for INPATIENT extractions only
df_reader = df_reader.loc[df_reader.type.isin(['ip'])]

In [ ]:
# Drop duplicates
df_reader = df_reader.loc[~df_reader.unique_id.duplicated()]

In [ ]:
# Reset index
df_reader = df_reader.reset_index(drop=True)

In [ ]:
# Save csv file 
df_reader.reset_index(drop=True).to_csv(write_dir+'inpatient_template_reader.csv')

## Parallelize survey extractions

In [ ]:
# Submit extraction jobs for each survey in df_reader
os.chdir(home_dir)
for i in np.arange((len(df_reader))):
    name_arg = 'extraction_' + str(i) + '_job'
    call = qsub(1, 9, name_arg, home_dir + 'ip_extraction_code.py',
           project = proj,
           script_args = [home_dir, write_dir, str(i), proj])
    call = " ".join(call)
    #print(call)
    os.system(call)

### Check to see which survey extractions broke

In [ ]:
# check to make sure I got all of them 
files = os.listdir(write_dir)
files = [x for x in files if ('num_visit' in x) | ('util_var' in x)]

# check to see which surveys failed
for y in df_reader.unique_id.unique():
    if len([x for x in files if str(y).lower() in x]) != 0:
        pass
    else:
        print(y)

In [ ]:
# use this to help locate extraction number, i (index) by NID
df_reader.loc[df_reader.nid == '151719']
# if above doesn't work, try NID as a string instead:
# i.e. df_reader.loc[df_reader.nid == '157635']

In [ ]:
# Re-run a single index (if needed)
i = 1202
name_arg = 'extraction_' + str(i) + '_job'
call = qsub(1,9,name_arg, home_dir + 'ip_extraction_code.py',
        project = proj,
        script_args = [home_dir, write_dir, str(i), proj], o = True)
call = " ".join(call)
#print(call)
os.system(call)

## Compile all surveys

In [ ]:
# compile_survey_data, see functions_to_prep_extractions.py
num_visit, fraction = compile_survey_data().compile_survey_data(df_reader = df_reader, 
             recall_rules = {4.3: [0,29.1],
                      52. :[29.1, 999]},
               path = write_dir)

In [ ]:
# collect surveys by type - either num_visit or fraction
data = {'num_visit':num_visit, 'fraction':fraction}

In [ ]:
for i in data.keys():
    temp = data[i]
    temp.loc[temp.location_id == 95, 'location_id'] = 4749
    temp.rename(columns = {'filepath': 'unique_id'}, inplace = True)
    
    # replace zero with tiny number
    temp.loc[temp['mean'] == 0, 'mean'] = .001
    
    ## print number of values below zero
    print(len(temp[temp['mean'] < 0]))
    temp = prep_upload(temp.copy(), recall_rules = {'cv_1_month_recall':[0, 29.1],
                                                    'cv_12_month_recall':[29.1, 999.]})
    temp['cv_marketscan'] = 0
    temp['cv_whs'] = 0
    temp['is_outlier'] = 0
    temp['cv_survey'] = 1
    temp['cv_whs'] = 0
    temp['cv_mics'] = 0
    data[i] = temp.copy()

In [ ]:
# create new data frames for separate survey types 
fraction = data['fraction']
num_visit = data['num_visit']

## Read in admin/facility data

In [ ]:
# pull in reported data
reported = pd.read_csv('FILEPATH')

reported['cv_survey'] = 0
reported['cv_1_month_recall'] = 0
reported['cv_12_month_recall'] = 0
reported['cv_marketscan'] = 0
reported['cv_whs'] = 0

reported['nid'] = pd.to_numeric(reported['nid'], downcast = 'float')
reported['is_outlier'] = 0
reported['visit_type'] = 'ip'

reported = reported.loc[['BRA' not in x for x in tabs.ihme_loc_id.tolist()]] 

In [ ]:
# Duplicate sources between "reported" file and clinical sources extracted
dup_nids = (3822,86886,86887,86888,86889,86890,86891,86892,86893,86894,86895,86896,86897,86898,86899,86900,
            86901,86902,86903,86904,86905,86906,86907,86908,86909,86910,86911,86912,86913,86914,86915,86916,
            86917,86997,86998,86999,87000,87001,87002,87003,87004,87005,87006,87007,87008,87009,87010,87011,
            90314,90315,90316,90317,90318,90319,90322,114876,121334,121405,121424,121425,121917,149501,149502,
            149503,149504,160484,205019,220786,220787,220788,220789,220790,220791,220792,220793,220794,220795,
            220796,220797,220798,220799,220800,237756,239353,293984)
reported = reported[~reported.nid.isin(dup_nids)]

In [ ]:
# Make sure those sources actually dropped 
reported.nid.value_counts()

In [ ]:
# Facility data 
fac_data = pd.read_csv('FILEPATH')
fac_data['is_outlier'] = 0
fac_data['cv_survey'] = 0
fac_data['cv_1_month_recall'] = 0
fac_data['cv_12_month_recall'] = 0
fac_data['cv_marketscan'] = 0
fac_data['cv_whs'] = 0
fac_data['visit_type'] = 'ip'

In [ ]:
# Older sources that are extracted as both sex, 0-100 age group - not great data 
fac_data = fac_data[fac_data.nid != 299087]
fac_data = fac_data[fac_data.nid != 299088]
fac_data = fac_data[fac_data.nid != 299089]
fac_data = fac_data[fac_data.nid != 299090]

In [ ]:
# Tabulated data 
tabs = pd.read_csv('FILEPATH')
tabs = tabs.loc[tabs.visit_type.isin(['ip_avg', 'ip_frac'])]
tabs['cv_marketscan'] = 0
tabs['cv_whs'] = 0
tabs['is_outlier'] = 0
tabs['cv_survey'] = 1

In [ ]:
# Read in re-extracted and newly extracted hospital/inpatient admin data for 2020
path = 'FILEPATH'                   
all_files = glob.glob(os.path.join(path, '*.csv'))     
ip_new_2020 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
ip_new_2020['is_outlier'] = 0
ip_new_2020['cv_survey'] = 0
ip_new_2020['cv_1_month_recall'] = 0
ip_new_2020['cv_12_month_recall'] = 0
ip_new_2020['cv_marketscan'] = 0
ip_new_2020['cv_whs'] = 0
ip_new_2020['visit_type'] = 'ip'
ip_new_2020.loc[ip_new_2020['sex_id'] == 1, "sex"] = 'Male'
ip_new_2020.loc[ip_new_2020['sex_id'] == 2, "sex"] = 'Female'

In [ ]:
# Time scale adjustment for under 1 age groups
ip_new_2020.loc[ip_new_2020['age_group_id'] == 2, 'mean'] = ip_new_2020['mean']/(365/7)
ip_new_2020.loc[ip_new_2020['age_group_id'] == 3, 'mean'] = ip_new_2020['mean']/(365/21)
ip_new_2020.loc[ip_new_2020['age_group_id'] == 388, 'mean'] = ip_new_2020['mean']/(365/154)
ip_new_2020.loc[ip_new_2020['age_group_id'] == 389, 'mean'] = ip_new_2020['mean']/(365/183)

In [ ]:
# Taiwan data
twn = pd.read_csv('FILEPATH')
twn['nid'] = 336203
twn['location_id'] = 8
twn['mean'] = pd.to_numeric(twn['val'])/pd.to_numeric(twn['pop'])
twn.rename(columns={'pop':'sample_size'}, inplace=True)
twn['cases'] = twn['val']
twn['lower'] = ''
twn['upper'] = ''
twn['standard_error'] = ''
twn['year_start'] = 2016
twn['year_end'] = 2016
twn = calc_se(twn.copy()).calc_se()
twn['is_outlier'] = 0
twn['visit_type'] = 'ip'
twn['cv_survey'] = 0
twn['cv_1_month_recall'] = 0
twn['cv_12_month_recall'] = 0
twn['cv_marketscan'] = 0
twn['cv_whs'] = 0

In [ ]:
# HCUP NIS data
nis = pd.read_csv('FILEPATH')
nis.rename(columns={'env':'mean'}, inplace=True)
nis.rename(columns={'population':'sample_size'}, inplace=True)
nis['cases'] = nis['admits']
nis['lower'] = ''
nis['upper'] = ''
nis['standard_error'] = ''
nis['year_start'] = nis['year_id']
nis['year_end'] = nis['year_id']
nis = calc_se(nis.copy()).calc_se()
nis['is_outlier'] = 0
nis['visit_type'] = 'ip'
nis['cv_survey'] = 0
nis['cv_1_month_recall'] = 0
nis['cv_12_month_recall'] = 0
nis['cv_marketscan'] = 0
nis['cv_whs'] = 0
nis.loc[nis['sex_id'] == 1, "sex"] = 'Male'
nis.loc[nis['sex_id'] == 2, "sex"] = 'Female'

## Combine all data types

In [ ]:
# set up for combining all data
all_cols = [] 
collect_data_all = pd.DataFrame()
dict_of_dfs = {'fracs_survey':fraction,
              'avg_survey':num_visit,
              'tabs':tabs,
              'reported':reported,
              'fac_data':fac_data,
              'ip_new_2020':ip_new_2020,
              'twn':twn,
              'nis':nis}

In [ ]:
# combine all data
for i in dict_of_dfs.keys():
    print(i)
    print(len(dict_of_dfs[i]))
    all_cols.extend(dict_of_dfs[i].columns.tolist())
    dict_of_dfs[i]['data_file'] = i
    dict_of_dfs[i]['sex'] = dict_of_dfs[i]['sex'].str.lower()
    collect_data_all = collect_data_all.append(dict_of_dfs[i])
collect_data_all.loc[collect_data_all.visit_type == 'ip', 'cv_12_month_recall'] = 1
collect_data_all.loc[collect_data_all.visit_type == 'ip', 'visit_type'] = 'ip_avg' 

In [ ]:
collect_data_all.loc[collect_data_all['age_group_id'] == 1, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 4, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 5, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 8, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 22, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 28, 'age_group_id'] = None
collect_data_all.loc[collect_data_all['age_group_id'] == 294, 'age_group_id'] = None

In [ ]:
collect_data_all.age_group_id.value_counts()

## Data cleanup pre-processing

In [ ]:
# adjust wrong iso code for romania
collect_data_all.loc[collect_data_all.ihme_loc_id == 'ROM', 'location_id'] = 52
collect_data_all.loc[collect_data_all.ihme_loc_id == 'ROM', 'ihme_loc_id'] = 'ROU'

In [ ]:
# start with no outliers
# modeling in 'continuous'
collect_data_all['is_outlier'] = 0
collect_data_all['measure'] = 'continuous'

In [ ]:
# check to make sure no blanks
try:
    print(len(collect_data_all.loc[collect_data_all.location_id == '']))
    collect_data_all = collect_data_all.loc[collect_data_all.location_id != '']
except:
    pass

In [ ]:
# convert nids to numerics
collect_data_all['location_id'] = pd.to_numeric(collect_data_all['location_id'])
collect_data_all['nid'] = pd.to_numeric(collect_data_all['nid'])

In [ ]:
# outlier Alaska admin data
collect_data_all.loc[collect_data_all['location_name'] == 'Alaska', 'is_outlier'] = 1

In [ ]:
# outliers not consistent with other data - don't want to process
collect_data_all.loc[(collect_data_all.nid == 112332) & (collect_data_all.sex == 'Female'), 'is_outlier'] = 1
collect_data_all.loc[(collect_data_all.nid == 20596) & (collect_data_all.sex == 'Female'), 'is_outlier'] = 1
collect_data_all.loc[(collect_data_all.ihme_loc_id == 'BIH') & (collect_data_all['mean'] >.5) & (collect_data_all.year_start == 2004), 'is_outlier'] = 1
collect_data_all.loc[collect_data_all.nid == 307778, 'is_outlier'] = 1
collect_data_all.loc[collect_data_all.nid == 138595, 'is_outlier'] = 1
collect_data_all.loc[(collect_data_all.location_id == 20) & (collect_data_all.year_start == 2002) & (collect_data_all.nid != 310156), 'is_outlier'] = 1
collect_data_all.loc[(collect_data_all.location_id == 180) & (collect_data_all.year_start == 2003) & (collect_data_all.nid != 310156), 'is_outlier'] = 1

In [ ]:
# GBR correct duplicate in lower uk levels
uk = collect_data_all.loc[collect_data_all.location_id == 95]
collect_data_all = collect_data_all.loc[collect_data_all.location_id != 95]
uk1 = uk.copy()
uk2 = uk.copy()
uk3 = uk.copy()
uk4 = uk.copy()
uk1['location_id'] = 4749 
uk2['location_id'] = 4636
uk3['location_id'] = 434
uk4['location_id'] = 433
uk = pd.concat((uk1, uk2, uk3, uk4))
collect_data_all = pd.concat((collect_data_all, uk))

In [ ]:
cols = pd.read_excel('FILEPATH').columns
cols = [x for x in cols if '.1' not in x]
cols.extend(['cv_12_month_recall', 'cv_survey', 'cv_1_month_recall', 'data_file', 'cv_sick','cv_mics', 'cv_whs', 'cv_marketscan', 'unique_id', 'true_recall'])            
cols = list(set(cols))
collect_data_all = collect_data_all[cols]

In [ ]:
# make sure these are numerics
collect_data_all['year_start'] = pd.to_numeric(collect_data_all['year_start'])
collect_data_all['year_end'] = pd.to_numeric(collect_data_all['year_end'])

In [ ]:
collect_data_all.loc[collect_data_all.year_start >= collect_data_all.year_end, 'year_start'] = collect_data_all.loc[collect_data_all.year_start >= collect_data_all.year_end, 'year_end']-1
collect_data_all['unit_value_as_published'] = 1

In [ ]:
# Calculate SE where it is missing (i.e. surveys)
collect_data_all = calc_se(collect_data_all.copy()).calc_se()

In [ ]:
# drop duplicate rows
collect_data_all = collect_data_all.drop(columns = ['index'])
collect_data_all = collect_data_all.drop_duplicates()

In [ ]:
# save here so don't have to re-run whole script when debugging
collect_data_all.to_csv('FILEPATH')

In [ ]:
# ONLY RUN THIS CELL IF NEEDED
#collect_data_all = pd.read_csv('FILEPATH')

In [ ]:
# check if there are any data types with 1-mo. recall & ip_avg (probably not)
len_check = len(collect_data_all.loc[((collect_data_all.visit_type == 'ip_avg') & (collect_data_all.cv_1_month_recall == 1))])

# if there are any of these, remove them
collect_data_all = collect_data_all.loc[~((collect_data_all.visit_type == 'ip_avg') & (collect_data_all.cv_1_month_recall == 1))] 

In [ ]:
# when surveys report average and fractions, only keep average data 
num_visit_loc_ids = num_visit.unique_id.unique().tolist()
print(len(collect_data_all.loc[((collect_data_all.data_file == 'fracs_survey') & (collect_data_all.unique_id.isin(num_visit_loc_ids)))]))
collect_data_all = collect_data_all.loc[~((collect_data_all.data_file == 'fracs_survey') & (collect_data_all.unique_id.isin(num_visit_loc_ids)))]

In [ ]:
# Merge on age_group_ids for data in correct age groups
# Read in age group metadata file
age_group_ids = pd.read_csv('FILEPATH')
# Rename columns to match collect_data_all names
age_group_ids = age_group_ids.rename(columns={'age_group_years_start':'age_start','age_group_years_end':'age_end'})
# Merge but make sure to keep all rows from collect_data_all
collect_data_all = pd.merge(collect_data_all, age_group_ids, on=['age_start', 'age_end'], how='left')
# Drop columns not needed
collect_data_all = collect_data_all.drop(columns=['age_group_name'])
# Clean up all columns
collect_data_all = collect_data_all[["nid","data_file","unique_id","location_name","ihme_loc_id",
             "sex","sex_issue","age_start","age_end","age_group_id","age_demographer","age_issue",
             "specificity","note_modeler","visit_type","group_review","group","year_start","year_end",
             "mean","cases","sample_size","measure","measure_issue","uncertainty_type_value",
             "design_effect","seq","cv_1_month_recall","smaller_site_unit","modelable_entity_name",
             "source_type","variable","cv_survey","extractor","cv_whs",
             "unit_value_as_published","page_num","true_recall","measure_adjustment","response_rate",
             "site_memo","is_outlier","uncertainty_type",
             "case_definition","cv_mics","cv_12_month_recall","cv_marketscan",
             "upper","lower","note_SR","data_sheet_filepath",
             "modelable_entity_id","recall_type","recall_type_value","effective_sample_size",
             "table_num","underlying_nid","case_diagnostics",
             "representative_name","urbanicity_type","sampling_type","case_name","standard_error",
             "field_citation_value","unit_type","year_issue","file_path",
             "underlying_field_citation_value","cv_sick","assign","location_id"]]

In [ ]:
# Additional file cleanup
for_split = collect_data_all.copy()
for_split['age_diff'] = for_split['age_end'] - for_split['age_start']
both_sex = for_split[for_split.sex == 'both']
both_sex_wide_age = both_sex[both_sex.age_diff >= 25]
both_sex_narrow_age = both_sex[both_sex.age_diff < 25]
male_female = for_split[for_split.sex != 'both']
mf_wide_age = male_female[male_female.age_diff >= 25]
mf_narrow_age = male_female[male_female.age_diff < 25]
both_sex_narrow_age['specificity'] = 'age'
mf_wide_age['specificity'] = 'sex'
for_split = pd.concat([both_sex_wide_age, both_sex_narrow_age, mf_wide_age, mf_narrow_age])

In [ ]:
for_split.sex.replace("male", "Male", inplace = True)
for_split.sex.replace("female", "Female", inplace = True)
for_split.sex.replace("both", "Both", inplace = True)

In [ ]:
for_split['seq'] = range(1, len(for_split) + 1)

In [ ]:
for_split.measure.value_counts()

In [ ]:
# Save file of post-processing/prep, pre-splitting, pre-xwalk version of data
for_split.to_csv('FILEPATH')

In [ ]:
# Read in sex-split file here
sex_split_df = pd.read_csv('FILEPATH')
# Check to make sure only M/F now
sex_split_df.sex.value_counts()

In [ ]:
# save files for crosswalk
sex_split_df.loc[(sex_split_df.cv_12_month_recall == 1) & (sex_split_df.cv_survey == 1) & (sex_split_df.visit_type.isin(['ip_avg']))].to_csv('FILEPATH')
sex_split_df.loc[(sex_split_df.cv_12_month_recall == 1) & (sex_split_df.visit_type.isin(['ip_frac']))].to_csv('FILEPATH')
sex_split_df.loc[(sex_split_df.cv_1_month_recall == 1) & (sex_split_df.visit_type.isin(['ip_frac']))].to_csv('FILEPATH')
sex_split_df.loc[sex_split_df.cv_survey == 0].to_csv('FILEPATH')